# Agents

An Agent is combination of an LLM model and some tools. The LLM act as brain of the Agent and Tools work as it's arms. 

In [5]:
## Setup LLM which we will use
from langchain_ollama.chat_models import ChatOllama
llm = ChatOllama(
    base_url = 'http://localhost:11434',
    model = 'qwen2.5:0.5b'
)

### Tools
Tools are like Python functions which can perform some action. 
  
**Note: LLM models cannot invoke a tool and give result. It is your responsibility to call tools. LLM can onlt tell which tools to be called**

In [4]:
### Lets try to create a simple Tool
from langchain.tools import tool

@tool
def my_first_tool(my_tool_arg: str) -> str:
    """This is description of Tool. And it is mandatory so that LLM can know what is purpose of the tool"""
    return "Some Output"

# Lets see information about these tools
print(f"{my_first_tool.name=}")
print(f"{my_first_tool.description=}")
print(f"{my_first_tool.args=}")

my_first_tool.name='my_first_tool'
my_first_tool.description='This is description of Tool. And it is mandatory so that LLM can know what is purpose of the tool'
my_first_tool.args={'my_tool_arg': {'title': 'My Tool Arg', 'type': 'string'}}


#### Lets try to create a sample tool

In [6]:
@tool
def big_crypto_coin(usd: float) -> float:
    "This tool converts the USD value to value for cyrpto currency names BIG and returns the amount in BIG currency"
    return 123.56*float(usd)

#### Lets bind the Tool with Our LLM

In [7]:
llm_with_crypto_tool = llm.bind_tools([big_crypto_coin])

In [10]:
# Lets ask the question and see what we get
result = llm_with_crypto_tool.invoke("How much BIG crypto can I get in 12 USD?")
print(result)

content='' additional_kwargs={} response_metadata={'model': 'qwen2.5:0.5b', 'created_at': '2025-05-22T15:57:43.766979053Z', 'done': True, 'done_reason': 'stop', 'total_duration': 405886742, 'load_duration': 15507290, 'prompt_eval_count': 185, 'prompt_eval_duration': 24054447, 'eval_count': 25, 'eval_duration': 361269271, 'model_name': 'qwen2.5:0.5b'} id='run--0ea15f71-c9cc-4499-984a-aef7840592e0-0' tool_calls=[{'name': 'big_crypto_coin', 'args': {'usd': 12}, 'id': '3906e7d6-1ab1-42b4-8992-f1c7d0c39b09', 'type': 'tool_call'}] usage_metadata={'input_tokens': 185, 'output_tokens': 25, 'total_tokens': 210}


As you can see, the LLM does not give us the answer. It provides a Tool call which we can use to get answer.

In [12]:
# And LLM suggest to make this Tool call
result.tool_calls

[{'name': 'big_crypto_coin',
  'args': {'usd': 12},
  'id': '3906e7d6-1ab1-42b4-8992-f1c7d0c39b09',
  'type': 'tool_call'}]

##### Lets try to make an automated way to make the tool calls. Since Tools calls is a list, we can loop over it and use the tool name and tool args to call tool

In [16]:
# First we need a mapping of Tool name with Tool's actual function so that we can call the function
# Followiung function can have more that 1 tool mapping, but we have only 1 right now. 
tool_map = {
    "big_crypto_coin": big_crypto_coin
}

for tool_info in result.tool_calls:
    tool = tool_map[tool_info['name']]
    tool_arg = tool_info['args']

    # Call the tool
    output = tool.invoke(tool_arg)
    print(output)

1482.72
